<a href="https://colab.research.google.com/github/dougnr89/Estudo_Roubo_Veiculos_SP/blob/main/TCC_Rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Bibliotecas**

In [1]:
!pip install squarify -qqq

import pandas as pd
import numpy as np
import seaborn as sns
import squarify
import glob
import re
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

import plotly.offline as py
py.init_notebook_mode(connected=True)

import plotly.graph_objs as go
import plotly.tools as tls
from folium import Map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

import warnings
warnings.filterwarnings("ignore")

**Import Dataset**

In [2]:
dir_path = "/content/"

def importaData(files):

    dados = dir_path + files
    allFiles = glob.glob('/content/*.csv')

    list_ = []
    cols = ['ANO_BO', 'DATAOCORRENCIA', 'PERIDOOCORRENCIA', 'BAIRRO', 'CIDADE', 'LATITUDE', 'LONGITUDE', 'LOGRADOURO', 'DESCRICAOLOCAL', 'DESCR_MARCA_VEICULO', 'DESCR_TIPO_VEICULO']

    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, delimiter=';', usecols=cols, encoding='ISO-8859-1')
        list_.append(df)

    df = pd.concat(list_, axis = 0, ignore_index = True)

    return df

all_data = importaData('*.csv')

**Visualizando os dados**


In [ ]:
all_data.tail()

**Estatística**

In [5]:
all_data.describe()

,ANO_BO
count,116599.000000
mean,2022.006244
std,0.080173
min,2021.000000
25%,2022.000000
50%,2022.000000
75%,2022.000000
max,2023.000000


**Pré Processamento**

In [6]:
print("Dados e Valores: ")

all_data.info()

Dados e Valores: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116599 entries, 0 to 116598
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ANO_BO               116599 non-null  int64 
 1   DATAOCORRENCIA       116599 non-null  object
 2   PERIDOOCORRENCIA     116599 non-null  object
 3   LOGRADOURO           110975 non-null  object
 4   BAIRRO               113449 non-null  object
 5   CIDADE               116521 non-null  object
 6   LATITUDE             107338 non-null  object
 7   LONGITUDE            107338 non-null  object
 8   DESCRICAOLOCAL       116599 non-null  object
 9   DESCR_MARCA_VEICULO  82346 non-null   object
 10  DESCR_TIPO_VEICULO   82219 non-null   object
dtypes: int64(1), object(10)
memory usage: 9.8+ MB


**Remoção duplicidade**

In [7]:
def remocaoduplicados(df):

    print('Quantidade duplicados: ', df.duplicated().sum())
    df.drop_duplicates(inplace=True)
    print('Quantidade duplicados após remoção: ', df.duplicated().sum())

    return df

all_data = remocaoduplicados(all_data)

Quantidade duplicados:  58248
Quantidade duplicados após remoção:  0


**Filtro Cidade SP**

In [8]:
def filtroSP(df):

    df.CIDADE.fillna('S.PAULO')
    df = df[df['CIDADE'] == 'S.PAULO']
    del df['CIDADE']

    return df

all_data = filtroSP(all_data)

**Ajuste String**

In [9]:
def lCase(df, cols):
    for col in cols:
        df[col] = df[col].str.title()

    return df

colsToLower = ["PERIDOOCORRENCIA", "LOGRADOURO", "BAIRRO", "DESCRICAOLOCAL", "DESCR_TIPO_VEICULO"]

all_data = lCase(all_data, colsToLower)

**Padrão Strings**

In [ ]:
def ajuste_string(string):
        string = string.replace('Avenida ', 'Av. ').replace('Av ', 'Av. ').replace('Rua ', 'R. ').replace('R ', 'R. ')                     .replace('R ', 'R. ').replace(',', '.').replace('Praça ', 'Pr. ').replace('Pr ', 'Pr. ')                     .replace('Pública', 'pública').replace('Jardim', 'Jd.').replace('Jd ', 'Jd. ')
        return string

all_data.DESCRICAOLOCAL = all_data.astype(str).DESCRICAOLOCAL.apply(str)
all_data.LOGRADOURO = all_data.astype(str).LOGRADOURO.apply(str)
all_data.LATITUDE = all_data.LATITUDE.str.replace(',','.').astype(float)
all_data.LONGITUDE = all_data.LONGITUDE.str.replace(',','.').astype(float)

**Categoria Veículo**

In [10]:
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "MOTONETA") | (all_data['DESCR_TIPO_VEICULO'] == "MOTOCICLO") | (all_data['DESCR_TIPO_VEICULO'] == "CICLOMOTO"),'TIPO_VEICULO'] = "MOTO"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "CAMINHONETE") | (all_data['DESCR_TIPO_VEICULO'] == "CAMIONETA") | (all_data['DESCR_TIPO_VEICULO'] == "UTILIT?RIO") | (all_data['DESCR_TIPO_VEICULO'] == "AUTOMOVEL"),'TIPO_VEICULO'] = "AUTOMOVEL"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "CAMINH?O") | (all_data['DESCR_TIPO_VEICULO'] == "CAMINH?O TRATOR") | (all_data['DESCR_TIPO_VEICULO'] == "SEMI-REBOQUE") | (all_data['DESCR_TIPO_VEICULO'] == "REBOQUE"),'TIPO_VEICULO'] = "CAMINHAO"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "MICRO-ONIBUS") | (all_data['DESCR_TIPO_VEICULO'] == "ONIBUS"),'TIPO_VEICULO'] = "ONIBUS"
all_data.loc[(all_data['TIPO_VEICULO'].isnull()) & (all_data['DESCR_TIPO_VEICULO'].notnull()),'TIPO_VEICULO'] = "OUTROS"


**Limpeza nome Veículo**

In [11]:
all_data.loc[(all_data['DESCR_MARCA_VEICULO'].str[0:2] == "I/") | (all_data['DESCR_MARCA_VEICULO'].str[0:2] == "H/") | (all_data['DESCR_MARCA_VEICULO'].str[0:2] == "R/"),'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str[2:]
all_data.loc[all_data['DESCR_MARCA_VEICULO'].str[0:4] == "IMP/",'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str[4:]
all_data.loc[:,'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str.replace("NOVO ","")
all_data.loc[:,'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str.replace("NOVA ","")


**Criando Marca e Modelo**

In [12]:
all_data['MARCA'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[0].str.split(" ", expand=True)[0]
all_data['MODELO'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[1].str.split(" ", expand=True)[0]
all_data.loc[all_data.MODELO == 'VW','MODELO'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[1].str.split(" ", expand=True)[1]


**Ajustando variações Marca**

In [13]:
all_data.loc[all_data['MARCA'] == "CHEV",'MARCA'] = "CHEVROLET"
all_data.loc[all_data['MARCA'] == "GM",'MARCA'] = "CHEVROLET"
all_data.loc[(all_data['MARCA'] == "VOLKS") | (all_data['MARCA'] == "VOLKSWAGEN") | (all_data['MARCA'] == "VOLKSWAGEM"),'MARCA'] = "VW"
all_data.loc[(all_data['MARCA'] == "M.B.") | (all_data['MARCA'] == "MBENZ") | (all_data['MARCA'] == "MERCEDES"),'MARCA'] = "M.BENZ"
all_data.loc[all_data['MARCA'] == "MMC",'MARCA'] = "MITSUBISHI"
all_data.loc[all_data['MARCA'] == "IVECOFIAT",'MARCA'] = "IVECO"
all_data.loc[(all_data['MARCA'] == "LR") | (all_data['MARCA'] == "LROVER"),'MARCA'] = "LAND ROVER"


**Dados Temporais**

In [15]:
def ajusteData(df, col = 'DATAOCORRENCIA'):
    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')

    indexs = []

    for row in df[col].items():
        if row[1] < pd.datetime(2010, 1, 1):
            indexs.append(row[0])

    df.drop(index=indexs, axis=1, inplace=True)

    df['DIA_SEMANA'] = df[col].dt.weekday
    df['MES_ANO'] = df[col].dt.strftime('%Y-%m')
    df['MES'] = df[col].dt.month

    df.drop(index=df[df['MES_ANO'] == 'NaT'].index, inplace=True, axis=0)

    return df

all_data = ajusteData(all_data)

**Análise Exploratória**

Visualizando DataFrame

In [17]:
all_data.head()

,ANO_BO,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,LATITUDE,LONGITUDE,DESCRICAOLOCAL,DESCR_MARCA_VEICULO,DESCR_TIPO_VEICULO,TIPO_VEICULO,MARCA,MODELO,DIA_SEMANA,MES_ANO,MES
0,2022,2021-12-31,A Noite,Rua Padre Pedro Rota,Sao Lucas,-23.592095,-46.551417,Via Pública,PEUGEOT 307 16 PR PK,Automovel,OUTROS,PEUGEOT,None,4,2021-12,12
1,2021,2021-12-31,A Noite,Rua Erva Botão,Jardim Dos Ipes,-23.502592,-46.414723,Outros,FORD/KA SE 1.0 HA C,Automovel,OUTROS,FORD,KA,4,2021-12,12
3,2022,2021-12-31,A Noite,Rua Antonio De Camargo Ortis,Itaim Paulista,-23.499228,-46.379238,Via Pública,HYUNDAI TUCSON GL 20L,Automovel,OUTROS,HYUNDAI,None,4,2021-12,12
5,2022,2021-12-31,A Noite,Avenida Sapopemba,Sapopemba,-23.610180,-46.484955,Via Pública,FIAT/PALIO ELX,Automovel,OUTROS,FIAT,PALIO,4,2021-12,12
7,2022,2021-12-31,A Noite,Avenida Almirante Delamare,Sacoma,-23.602919,-46.597583,Via Pública,NISSAN/KICKS ACTIVE CVT,Automovel,OUTROS,NISSAN,KICKS,4,2021-12,12


In [18]:
df_ts = all_data.groupby('DATAOCORRENCIA').count()["ANO_BO"].to_frame().rename(columns={"ANO_BO": "OCORRENCIAS"})

trace = [go.Scatter(x = df_ts.index, y = df_ts.OCORRENCIAS)]

layout = dict(
    title='Ocorrências Roubo de Veículo',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=12,
                     label='1yr',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=trace, layout=layout)
py.iplot(fig)

In [19]:
ocorrencias = all_data.groupby('MES_ANO').count()['ANO_BO'].reset_index()
ocorrencias.rename(columns={'MES_ANO': 'Data', 'ANO_BO':'Ocorrências Roubo Veículo'}, inplace=True)

trace = [go.Scatter(x = ocorrencias.Data, y = ocorrencias['Ocorrências Roubo Veículo'])]

layout = dict(
    title='Ocorrências Roubo de Veículo Mês',
    yaxis=dict(
    title='Ocorrências'
    ),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=12,
                     label='1yr',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=trace, layout=layout)
py.iplot(fig)

Podemos notar uma tendência de aumento no número de ocorrência de roubo de veículo. Alguns dos fatores que podem explicar esse aumento são:

- Parada em semáforo de regiões classificadas como perigosas.
- Utilização de ruas com falta de iluminação, pouco ou nenhum policiamento e sinal de telefonia fraco.
- Objetos deixados em veículos.

**Variações ocorrências por ano**

In [ ]:
all_data.ANO_BO = all_data.ANO_BO.astype(int)
ocorrencias_ano = all_data.groupby('ANO_BO').count()['DATAOCORRENCIA'].reset_index()
ocorrencias_ano['VARIACAO'] = 0
ocorrencias_ano['VARIACAO_ACUMULADA'] = 0

for i in range (0, 8):
    ocorrencias_ano.iloc[i+1, 2] = round((1 - (ocorrencias_ano.iloc[i, 1] / ocorrencias_ano.iloc[i+1, 1])) * 100, 2)
    ocorrencias_ano.iloc[i+1, 3] = round((ocorrencias_ano.iloc[i, 3] + ocorrencias_ano.iloc[i+1, 2]), 2)

ocorrencias_ano.VARIACAO = ocorrencias_ano.VARIACAO.astype(str) + ' %'
ocorrencias_ano.VARIACAO_ACUMULADA = ocorrencias_ano.VARIACAO_ACUMULADA.astype(str) + ' %'

ocorrencias_ano

In [ ]:
data = [
    go.Bar(
        x = ocorrencias_ano.ANO_BO.values[1:],
        y = ocorrencias_ano.VARIACAO.values[1:],
        marker = dict(
          color = ['green', 'green', 'green', 'green', 'red', 'green', 'red', 'red']
        ),
        name = 'Variação com ano anterior'
    ),
    go.Scatter(
        x = ocorrencias_ano.ANO_BO.values[1:],
        y = ocorrencias_ano.VARIACAO_ACUMULADA.values[1:],
        marker = dict(
          color = 'blue'
        ),
        name = 'Variação acumulada'
    )
]

layout = dict(
    title='Variação do Número de Ocorrências de Roubo de Veículo',
    yaxis=dict(
        title='Variação (%)',
        ),
    )

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
all_data['diasemana'] = pd.Categorical(all_data['diasemana'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)

trace = [go.Bar(
            x = all_data['diasemana'].value_counts().sort_index().index,
            y = all_data['diasemana'].value_counts().sort_index().values,
            textposition = 'auto',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1,
                    )
                ),
            opacity=0.8
)]

layout = go.Layout(
    title='Ocorrências de Roubo de Veículo na Semana (2019 - 2022)',
    yaxis=dict(
        title='Ocorrências'
        ),
)

fig = go.Figure(data=trace, layout=layout)

py.iplot(fig)

In [ ]:
colors = ['#170B3B', '#A9D0F5', '#08088A', '#DBA901']

trace = [go.Pie(labels=all_data.PERIDOOCORRENCIA.unique(), values=all_data.PERIDOOCORRENCIA.value_counts().values,
               hoverinfo='label+percent',
               textfont=dict(size=18),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))]

layout = go.Layout(
    title='Período das Ocorrências de Roubo de Veículo (2019 - 2022)'
)

fig = go.Figure(data = trace, layout = layout)

py.iplot(fig)

Filtro dados 2022

In [ ]:
data = all_data[all_data["ANO_BO"] == 2018].copy()

In [ ]:
trace = [go.Bar(
            y = data['DESCRICAOLOCAL'].value_counts().index,
            x = data['DESCRICAOLOCAL'].value_counts().values,
            orientation='h',
            textposition = 'auto',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1,
                    )
                ),
            opacity=0.8
)]

layout = go.Layout(
    height=800,
    title='Locais das Ocorrências de Roubo de Veículo em 2022',
    xaxis=dict(
        title = 'Ocorrências',
        type='log',
        ),
     yaxis=go.layout.YAxis(
     automargin=True
     )
)

fig = go.Figure(data=trace, layout=layout)

py.iplot(fig)

As vias públicas são os locais com maiores indíces de roubo de veículo.

In [ ]:
data_ = data[(data['DESCR_TIPO_VEICULO'] != 'Inexist.') & (data['DESCR_TIPO_VEICULO'] != 'Não Informado')]

trace = [go.Bar(
            x = data_["DESCR_TIPO_VEICULO"].value_counts().index,
            y = data_["DESCR_TIPO_VEICULO"].value_counts().values,
            text=data_["DESCR_TIPO_VEICULO"].value_counts().values,
            textposition = 'auto',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1,
                    )
                ),
            opacity=0.8
)]

layout = go.Layout(
    title='Tipos de veículos mais roubados em 2022',
    yaxis=dict(
        title='Ocorrências',
        ),
)

fig = go.Figure(data=trace, layout=layout)

py.iplot(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))

cmap = matplotlib.cm.coolwarm
mini = data.LOGRADOURO.value_counts().values[19]
maxi = data.LOGRADOURO.value_counts().values[0]
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in data.LOGRADOURO.value_counts().iloc[:100].values]

squarify.plot(sizes=data.LOGRADOURO.value_counts().iloc[:20].values, label=data.LOGRADOURO.value_counts().iloc[:20].index, color=colors)

plt.axis('off')
plt.title("Locais com mais ocorrências de roubo de veículo em 2022", fontsize=18)
ttl = ax.title
ttl.set_position([.5, 1.05])

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))

cmap = matplotlib.cm.coolwarm
mini = data.BAIRRO.value_counts().values[49]
maxi = data.BAIRRO.value_counts().values[0]
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in data.BAIRRO.value_counts().iloc[:100].values]

squarify.plot(sizes=data.BAIRRO.value_counts().iloc[:50].values, label=data.BAIRRO.value_counts().iloc[:50].index, color=colors)

plt.axis('off')
plt.title("Bairros com mais ocorrências de roubo de veículo em 2022", fontsize=18)
ttl = ax.title
ttl.set_position([.5, 1.05])

**Criando colunas coordenadas**

In [ ]:
df = data.dropna(subset=['LATITUDE', 'LONGITUDE']).copy()
df = df[df['DATAOCORRENCIA'] > "2022-01-01"]
df['coord'] = df[['LATITUDE','LONGITUDE']].apply(tuple, axis=1)

**Mapa de calor do roubo de veículo**

In [ ]:
m = Map([-23.64992751, -46.62569322], zoom_start=10, width='100%', height='100%')
m.add_child(HeatMap(df['coord'], radius=8))
m

**ClusterMap do roubo de veículos**

In [ ]:
pointMap = Map([-23.550, -46.620], zoom_start=10.5, width='100%', height='100%')
pointMap.add_child(FastMarkerCluster(df['coord']))
pointMap

**Prevendo Roubo de Veículos**

**Modelagem Arima**

Série temporal

In [ ]:
ocorrencias.index =  ocorrencias.Data
ocorrencias.drop(columns='Data', inplace=True)
ocorrencias.index = pd.to_datetime(ocorrencias.index)

Biblioteca

In [ ]:
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose

**Decomposição série Temporal**

In [ ]:
result = seasonal_decompose(ocorrencias, model='multiplicative')
g = result.plot()
g.set_figwidth(14)
g.set_figheight(14)

Biblioteca

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

**Plot Correlação ACF**

In [ ]:
g = plot_acf(ocorrencias, title="ACF Correlation")
g.set_figheight(8)
g.set_figwidth(14)

Podemos começar diferenciando a série temporal com ordem d = 1, a fim de obter uma série temporal estacionária. Se a série não representar bons resultados, outras diferenciações podem ser necessarias.

Diferenciando série Temporal

In [ ]:
ocorrencias_diff = ocorrencias.diff(periods=1)
ocorrencias_diff = ocorrencias_diff[1:]

Plot da série temporal diferenciada

In [ ]:
g = ocorrencias_diff.plot(figsize=(14,8), title="Série Temporal Diferenciada", legend=False)

Plot Correlação ACF

In [ ]:
g = plot_acf(ocorrencias_diff, title="ACF Correlation (d = 1)")
g.set_figheight(8)
g.set_figwidth(14)

Através do ACF plot, podemos ver que o valor q = 13, pode ser uma boa escolha de parâmetro.

Definindo combinações

In [ ]:
pdq = [(p, 1, 13) for p in range(0, 4)]

DEFINE P e Q ENTRE 0 e 3

In [ ]:
P = Q = range(0, 4)

DEFINE COMBINAÇÕES ENTRE P, D e Q

In [ ]:
seasonal_pdq = [(x[0], 1, x[1], 12) for x in list(itertools.product(P, Q))]

Melhores parâmetros para o modelo

In [ ]:
#scores = {}

#for param in pdq:
#    for param_seasonal in seasonal_pdq:
#        try:
#            mod = sm.tsa.statespace.SARIMAX(ocorrencias,
#                                            order=param,
#                                            seasonal_order=param_seasonal,
#                                            enforce_stationarity=False,
#                                            enforce_invertibility=False)
#
#            results = mod.fit()
#            scores[param, param_seasonal] = results.aic
#
#        except:
#            continue
#
#print("Melhores parâmetros: ", min(scores, key=scores.get)," AUC: ", min(scores.values()))

# Melhores parâmetros: ((3, 1, 13), (3, 1, 0, 12)) AUC: 916.5604661162852

Modelo com melhores parâmetros

In [ ]:
mod = sm.tsa.statespace.SARIMAX(ocorrencias,
                                order=(3, 1, 13),
                                seasonal_order=(3, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

Resumo modelo

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

Validação

Validação do Modelo - Prevendo ocorrências a partir de 2020

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2020-01-01'), dynamic=False)
pred_ci = pred.conf_int()

ax = ocorrencias['2019':].plot(figsize=(14, 8))
pred.predicted_mean.plot(ax=ax, label='Previsão', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_title("Previsão de Ocorrências de Roubo de Veículo", fontsize=18)
ax.set_xlabel('Data')
ax.set_ylabel('Ocorrências de Roubo de Veículo')
plt.legend()

plt.show()

Calculando o MSE

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = ocorrencias["2020-01-01":].squeeze()

mse = ((y_forecasted - y_truth) ** 2).mean()

print('Mean Squared Error {}'.format(round(mse)))

**Previsão**

Previsão de ocorrências próximos 3 anos

In [ ]:
pred_uc = results.get_forecast(steps=36)
pred_ci = pred_uc.conf_int()

ax = ocorrencias['2022':].plot(figsize=(14, 8))
pred_uc.predicted_mean.plot(ax=ax, label='Previsão')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)

ax.set_title("Previsão de Ocorrências de Roubo de Veículo", fontsize=18)
ax.set_xlabel('Data')
ax.set_ylabel('Ocorrências de Roubo de Veículo')

plt.legend()
plt.show()

A previsão realizada pelo modelo ARIMA reforça a tendência histórica de crescimento dos roubos de veículos, com um intervalo de confiança relativamente alto, devido as variações encontradas na série temporal.